# Pivotality: The influence of whales in DAO Governance | Part 2

## Definition

On a given proposal, whales are considered _pivotal voters_ when, taken together, casting their votes altered the result. [[1]](https://www.researchgate.net/publication/249676423_Pivotal_Voting) [[2]](https://publications.ut-capitole.fr/id/eprint/15307/1/PivotProbabilitiesMay2014ShortVersionR.pdf).

## Aims of this report

This article aims to analyze the top 60 DAOs, ranked by [treasury size](https://deepdao.io/) to determine the impact of large token holders (whales) on their governance.

On a previous article, we looked at off-chain governance (via Snapshot) to calculate our **_whale pivotality_ metric**. In this article, we will be conducting this study on on-chain governance.

Again, we define whales as the **top 5% voters in terms of voting power** for a given proposal (voters with voting power at or above the 95th percentile of voting power for that proposal). 


## Takeaways

- Compared to off-chain governance on Snapshot, on-chain Governance has very low participation per-proposal
- only 1 organization had a whale pivotality above 5% (Arbitrum, which only has two nonsense, test-proposals at the time of writing)
- 3 other DAOs had a whale pivotality between 2% and 4%, the remaining with 0%

## Calculations

Below, a brief summary of calculations performed to build the whale pivotality statistics.

See the [repository](https://github.com/butterymoney/gov_analysis) for more details.

In [1]:
# sets up the pynb environment
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.dataframe_filters.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
    get_number_of_voters_per_proposal,
)

### Load data

Load each voter's choice and voting power for up to the last one hundred proposals in each DAO.

In [2]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_tally_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_tally_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [4]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(organization_name), space_proposals)
            for organization_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_organization_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [5]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)

voter_counts = get_number_of_voters_per_proposal(plutocracy_report_data)

### Compute score differences

For each choice of each proposal, get:

- Actual score.
- Hypothetical score that would have been produced if whales didn't vote.

Then compute if the outcome is different, meaning if the outcome would have been changed if whales didn't vote.

Then compute, for each DAO, the changed outcome proportion among the last 100 proposals.

In [6]:
score_differences_dfs = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "proposal_id",
                "proposal_title",
                "proposal_start",
                "proposal_end",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "outcome_old",
                "outcome_new"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["whale_vp_proportion","total_vp"], ascending=False)

        try:
            changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]
        except KeyError:
            changed_outcome_proportions[organization] = 0

        organization_id = plutocracy_report_data[organization].iloc[0]["proposal_organization_id"]
        organization_score_diff_df = score_differences_dfs[organization]

        organization_score_diff_df["total_vp"] = organization_score_diff_df["total_vp"].apply("{:.9f}".format)

        organization_score_diff_df.style.format({"whale_vp_proportion": "{:.2%}".format})

        organization_score_diff_df["voter_count"] = voter_counts[organization]
        organization_score_diff_df["total_vp"] = organization_score_diff_df["total_vp"].astype("float")

        sort_key = organization_score_diff_df.loc[:, ["voter_count", "total_vp"]]
        sort_key["voter_count_rank"] = sort_key.loc[:,"voter_count"].sort_values(ascending=False).rank(method="max", ascending=False)
        sort_key["total_vp_rank"] = sort_key.loc[:,"total_vp"].sort_values(ascending=False).rank(method="dense", ascending=False)

        organization_score_diff_df["rank"] = sort_key.apply(lambda row: row["voter_count_rank"] + row["total_vp_rank"], axis=1)
        organization_score_diff_df.sort_values(
            "rank",
            inplace=True
        )
        
        organization_score_diff_df.index = organization_score_diff_df.index.to_series().apply(
            lambda s: f'<a href=http://snapshot.org/#/{organization_id}/proposal/{s} rel="noopener noreferrer" target="_blank">{s}</a>'
        )


changed_outcome_proportions_raw = changed_outcome_proportions.copy()
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.0%}".format(proportion)
)

In [7]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)

### Synthesis

For each DAO, show the percentage of proposals, the outcome of which would have changed if whales didn't vote (whale pivotality).

In [8]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, "whale pivotality", changed_outcome_proportions)
dao_overview

,# of whales,all voters,whale pivotality
DAO,,,
Aave,109,5274,4%
Ampleforth,2,109,0%
Angle,1,5,0%
Arbitrum Core,1558,31127,100%
Arbitrum Treasury,1785,34833,100%
ENS,4,1853,0%
Gitcoin,28,2125,2%
Hop,7,329,0%
InstaDapp,1,28,0%


## Case studies

### Aave

#### Proportion of Outcomes Changed:

In [9]:
print(f"{changed_outcome_proportions['Aave']} of Aave's proposal outcomes change after filtering out whale voting power.")

4% of Aave's proposal outcomes change after filtering out whale voting power.


#### Proposal Analysis:

For example, [this proposal](https://www.tally.xyz/gov/aave/proposal/193) to freeze out some low liquidity assets on Aave V2 AMM: G-UNI DAI/USDC and G-UNI USDC/USDT.

In [10]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '193'].iloc[0]['proposal_choices']
proposal_score_differences = score_differences_dfs["Aave"][score_differences_dfs["Aave"]["proposal_id"] == "193"]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '193'].iloc[0]['proposal_scores']

non_whales = [
    x - y for x, y in zip(eval(proposal_scores), proposal_score_differences)
]

number_format = "{:.1f}".format

pd.DataFrame(
    {choice: [score, score_diff, non_whale_score] for choice, score, score_diff, non_whale_score in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences, non_whales)},
    index=["Scores", "Whale-only scores", "Non-whale scores"],
).style.format(number_format)

,FOR,AGAINST,ABSTAIN
Scores,17873201996188790784.0,226512566038205309124608.0,0.0
Whale-only scores,0.0,226512181369205327659008.0,0.0
Non-whale scores,17873201996188790784.0,384668999981465600.0,0.0


99.9% of voting power was attributed to whales, with 99.9% of proposal voting power allocated to voting for the proposal not to pass.

We also observe that non-whale voting power is very low in this proposal. Six unique addresses voted for this proposal, eleven against.

### Gitcoin

#### Proportion of Outcomes Changed:

In [11]:
print(f"{changed_outcome_proportions['Gitcoin']} of Gitcoin's proposal outcomes change after filtering out whale voting power.")

2% of Gitcoin's proposal outcomes change after filtering out whale voting power.


#### Proposal Analysis:

This defeated proposal to ratify the Fraud Defence and Detection budget for Gitcoin season 13 is the only proposal whose outcome changes after filtering out whales. Technically, it was "defeated" due to not meeting the required quorum

In [13]:
propsal_choices = plutocracy_report_data['Gitcoin'][plutocracy_report_data['Gitcoin']['proposal_id'] == '17'].iloc[0]['proposal_choices']
proposal_score_differences = score_differences_dfs["Gitcoin"][score_differences_dfs["Gitcoin"]["proposal_id"] == '17']["score_differences"][0]
proposal_scores = plutocracy_report_data['Gitcoin'][plutocracy_report_data['Gitcoin']['proposal_id'] == '17'].iloc[0]['proposal_scores']

non_whales = [
    x - y for x, y in zip(eval(proposal_scores), proposal_score_differences)
]


pd.DataFrame(
    {choice: [score, score_diff, non_whale_score] for choice, score, score_diff, non_whale_score in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences, non_whales)},
    index=["Scores", "Whale-only scores", "Non-whale scores"],
).style.format(number_format)

,FOR,AGAINST,ABSTAIN
Scores,320966599999999967232.0,27000000000000000000.0,0.0
Whale-only scores,320966599999999967232.0,0.0,0.0
Non-whale scores,0.0,27000000000000000000.0,0.0
